<a href="https://colab.research.google.com/github/kush980/Image-Classification-Web-app/blob/master/Image_Classification_Web_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [5]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset
%matplotlib inline

#Importing and creating custom Dataset

In [ ]:
!wget https://raw.githubusercontent.com/nirajpandkar/flowers-classification-pytorch/master/cat_to_name.json
!wget https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz
!mkdir data && tar -xvf flower_data.tar.gz -C data

In [3]:
data_dir = 'data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [6]:
import os,json

class FlowersDataset(ImageFolder):
    def __init__(self, root, transform):
        self.root = root
        super().__init__(self.root)
        self.files = [folder+"/"+fname for folder in os.listdir(root) for fname in os.listdir(root+"/"+folder) if fname.endswith('.jpg')]
        self.transform = transform
        with open('cat_to_name.json', 'r') as f:
          self.classes_idx = json.load(f)
        category_map = sorted(self.classes_idx.items(), key=lambda x: int(x[0]))
        self.classes = [cat[1] for cat in category_map]
    
    def __len__(self):
        return len(self.files)

In [7]:
import torchvision.transforms as T

img_size = 224
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [10]:
train_dataset = FlowersDataset(train_dir, T.Compose([T.Resize(img_size), 
                                           T.Pad(8, padding_mode='reflect'),
                                           T.RandomCrop(img_size), 
                                           T.ToTensor(), 
                                           T.Normalize(*imagenet_stats)]))
print(train_dataset.classes)
print(len(train_dataset.classes))
print(len(train_dataset.files))

['pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea', 'english marigold', 'tiger lily', 'moon orchid', 'bird of paradise', 'monkshood', 'globe thistle', 'snapdragon', "colt's foot", 'king protea', 'spear thistle', 'yellow iris', 'globe-flower', 'purple coneflower', 'peruvian lily', 'balloon flower', 'giant white arum lily', 'fire lily', 'pincushion flower', 'fritillary', 'red ginger', 'grape hyacinth', 'corn poppy', 'prince of wales feathers', 'stemless gentian', 'artichoke', 'sweet william', 'carnation', 'garden phlox', 'love in the mist', 'mexican aster', 'alpine sea holly', 'ruby-lipped cattleya', 'cape flower', 'great masterwort', 'siam tulip', 'lenten rose', 'barbeton daisy', 'daffodil', 'sword lily', 'poinsettia', 'bolero deep blue', 'wallflower', 'marigold', 'buttercup', 'oxeye daisy', 'common dandelion', 'petunia', 'wild pansy', 'primula', 'sunflower', 'pelargonium', 'bishop of llandaff', 'gaura', 'geranium', 'orange dahlia', 'pink-yellow dahlia', 'cau

In [11]:
validation_dataset = FlowersDataset(valid_dir, T.Compose([T.Resize(img_size), 
                                           T.Pad(8, padding_mode='reflect'),
                                           T.RandomCrop(img_size), 
                                           T.ToTensor(), 
                                           T.Normalize(*imagenet_stats)]))

test_dataset = FlowersDataset(test_dir, T.Compose([T.Resize(img_size), 
                                           T.Pad(8, padding_mode='reflect'),
                                           T.RandomCrop(img_size), 
                                           T.ToTensor(), 
                                           T.Normalize(*imagenet_stats)]))